In [3]:
import os
import csv
import pandas as pd
import numpy as np
import random
import scipy.stats as stats
import json
from glob import glob
import re
import skimage as io
import matplotlib.pyplot as plt

In [316]:
#Create a csv with the initial columnnames
import csv
with open('database/smFISH_database.csv', 'w', newline='') as table:
    fieldnames = ["filename",'folder',"target", "count", ]
    writer = csv.DictWriter(table, fieldnames=fieldnames)
    writer.writeheader()
    

In [317]:
# This is the path where you want to search
path ='/Volumes/mix_files/neuron_smFISH/best_results'
csv_file= '/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/smFISH_database.csv'


# this is the extension you want to detect
extension = '.csv'

length_of_file = []



for root, dirs_list, files_list in os.walk(path):
    for file_name in files_list:
        if file_name[-4:] == extension and not file_name.startswith('._'):
            #print(os.path.join(root, file_name))
            reader = pd.read_csv(os.path.join(root, file_name))
            length_of_file = len(reader)
           
            with open(csv_file, 'a', newline='') as file:
                fieldnames = ["filename","folder","target","count"]
                writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=',',lineterminator='\n')
                writer.writerow({"filename": file_name, "folder": root, "count" : length_of_file })

     

In [318]:
df = pd.read_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/smFISH_database.csv')

In [319]:
df['folder'] = df["folder"].str.replace("/Volumes/mix_files/neuron_smFISH/best_results/","")

In [320]:
#df.loc[df['folder'].str.contains('Hbs1l'), 'folder'] = 'Scram'

In [321]:
df.head()

,filename,folder,target,count
0,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,194
1,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,867
2,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,379
3,20211123_smiFISH_HBS1LshRNA_04_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,554
4,20211123_smiFISH_HBS1LshRNA_05_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,480


In [322]:
# Create a column with the smFISH label and structure in the cell

#df['target'] =  df.folder.str[:8] 
df.loc[df['folder'].str.contains('neurite'), 'structure'] = 'neurite'
df.loc[df['folder'].str.contains('soma'), 'structure'] = 'soma'

#df['structure'] = df["folder"].str.replace('smFISH-\d_',"")


In [323]:
# Create a column with the gene target
#df.loc[df['target']== 'smFISH-1', 'gene'] = 'Map2'
#df.loc[df['target']== 'smFISH-2', 'gene'] = 'GAPDH'

df.loc[df['folder'].str.contains('Map2'), 'gene'] = 'Map2'
df.loc[df['folder'].str.contains('GFP'), 'gene'] = 'GFP'
df.loc[df['folder'].str.contains('Kif21B'), 'gene'] = 'Kif21B'
df.loc[df['folder'].str.contains('Cf'), 'gene'] = 'Cf'

df.loc[df['folder'].str.contains('smFISH-2'), 'gene'] = 'GAPDH'



df.loc[df['filename'].str.contains('Scram'), 'shRNA'] = 'Scram'
df.loc[df['filename'].str.contains('HBS1L'), 'shRNA'] = 'HBS1L'
df.loc[df['filename'].str.contains('Ago2'), 'shRNA'] = 'Ago2'


In [324]:
#sort the database
df.sort_values(
    by=["shRNA", "gene", "structure"])

df.head(5)

,filename,folder,target,count,structure,gene,shRNA
0,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,194,soma,GAPDH,HBS1L
1,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,867,soma,GAPDH,HBS1L
2,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,379,soma,GAPDH,HBS1L
3,20211123_smiFISH_HBS1LshRNA_04_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,554,soma,GAPDH,HBS1L
4,20211123_smiFISH_HBS1LshRNA_05_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,480,soma,GAPDH,HBS1L


In [325]:
len(df)

136

In [326]:
df.to_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/smFISH_database_2.csv', index=False)  


In [327]:
#read in the area of the structures

#Create a csv with the initial columnnames
import csv
with open('database/area_sizes.csv', 'w', newline='') as table:
    fieldnames = ["filename",'area', "folder"]
    writer = csv.DictWriter(table, fieldnames=fieldnames)
    writer.writeheader()


In [328]:
# This is the path where you want to search
path_area ='/Volumes/mix_files/neuron_smFISH/masks/'
area_file= '/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/area_sizes.csv'


# this is the extension you want to detect
extension = '.csv'

size_of_mask = []



for root, dirs_list, files_list in os.walk(path_area):
    for file_name in files_list:
        if file_name[-4:] == extension and not file_name.startswith('._'):
            #print(os.path.join(root, file_name))
            reader = pd.read_csv(os.path.join(root, file_name))
            size_of_mask = float(reader['Area'])
           
            with open(area_file, 'a', newline='') as file:
                fieldnames = ["filename",'area', "folder"]
                writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=',',lineterminator='\n')
                writer.writerow({"filename": file_name, "folder": root, "area":  size_of_mask   })

     

In [329]:
area_csv = pd.read_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/area_sizes.csv')

area_csv.loc[area_csv['filename'].str.contains('neurite'), 'structure'] = 'neurite'
area_csv.loc[area_csv['filename'].str.contains('soma'), 'structure'] = 'soma'
area_csv.loc[area_csv['filename'].str.contains('Neurite'), 'structure'] = 'neurite'
area_csv.loc[area_csv['filename'].str.contains('Soma'), 'structure'] = 'soma'
#area_csv['structure'] = area_csv["folder"].str.replace("/Volumes/mix_files/neuron_smFISH/masks/","")

area_csv.sort_values(
    by=["folder", "filename"])
area_csv.head(5)

,filename,area,folder,structure
0,20220426_Ago2CfGFP-01_2022-04-26_Sam_smifish_C...,477395.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite
1,20220426_Ago2CfGFP-02_2022-04-26_Sam_smifish_C...,263793.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite
2,20220426_Ago2CfGFP-03_2022-04-26_Sam_smifish_C...,0.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite
3,20220426_Ago2CfGFP-04_2022-04-26_Sam_smifish_C...,251333.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite
4,20220426_Ago2CfGFP-05_2022-04-26_Sam_smifish_C...,127864.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite


In [330]:
# Create a column with the gene target
area_csv.loc[area_csv['filename'].str.contains('Map2'), 'gene'] = 'Map2'
#area_csv.loc[area_csv['filename'].str.contains('smiFISH'), 'gene'] = 'Map2'
area_csv.loc[area_csv['filename'].str.contains('GFP'), 'gene'] = 'GFP'
area_csv.loc[area_csv['filename'].str.contains('Kif21B'), 'gene'] = 'Kif21B'
area_csv.loc[area_csv['filename'].str.contains('Cf'), 'gene'] = 'Cf'

area_csv.loc[area_csv['filename'].str.contains('smFISH-2'), 'gene'] = 'GAPDH'



area_csv.loc[area_csv['filename'].str.contains('Scram'), 'shRNA'] = 'Scram'
area_csv.loc[area_csv['filename'].str.contains('HBS1L'), 'shRNA'] = 'HBS1L'
area_csv.loc[area_csv['filename'].str.contains('Ago2'), 'shRNA'] = 'Ago2'


area_csv.sort_values(
    by=["shRNA", "gene", "structure"])
area_csv.head(5)


area_csv.head(5)

,filename,area,folder,structure,gene,shRNA
0,20220426_Ago2CfGFP-01_2022-04-26_Sam_smifish_C...,477395.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2
1,20220426_Ago2CfGFP-02_2022-04-26_Sam_smifish_C...,263793.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2
2,20220426_Ago2CfGFP-03_2022-04-26_Sam_smifish_C...,0.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2
3,20220426_Ago2CfGFP-04_2022-04-26_Sam_smifish_C...,251333.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2
4,20220426_Ago2CfGFP-05_2022-04-26_Sam_smifish_C...,127864.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2


In [331]:
len(area_csv)

216

In [332]:
### Combine the two datasets
#df['id'] = df["shRNA"] + '_' +  df["gene"] + '_' + df["structure"]
#df.head(5)
#area_csv['id'] = area_csv["shRNA"] + '_' +  area_csv["gene"] + '_' + area_csv["structure"]
#area_csv.head(5)
#area_csv.to_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/area_sizes_2.csv', index=False)  


In [333]:
### Combine the two datasets
split_word = 'FusionStitcher'
df['id'] = df.filename.str.partition(split_word)[0]
df.head()
df.to_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/smFISH_database_3.csv', index=False)  


In [334]:
area_csv['id'] = area_csv['filename'].str.partition(split_word)[0]
area_csv.head()

,filename,area,folder,structure,gene,shRNA,id
0,20220426_Ago2CfGFP-01_2022-04-26_Sam_smifish_C...,477395.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2,20220426_Ago2CfGFP-01_2022-04-26_Sam_smifish_C...
1,20220426_Ago2CfGFP-02_2022-04-26_Sam_smifish_C...,263793.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2,20220426_Ago2CfGFP-02_2022-04-26_Sam_smifish_C...
2,20220426_Ago2CfGFP-03_2022-04-26_Sam_smifish_C...,0.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2,20220426_Ago2CfGFP-03_2022-04-26_Sam_smifish_C...
3,20220426_Ago2CfGFP-04_2022-04-26_Sam_smifish_C...,251333.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2,20220426_Ago2CfGFP-04_2022-04-26_Sam_smifish_C...
4,20220426_Ago2CfGFP-05_2022-04-26_Sam_smifish_C...,127864.0,/Volumes/mix_files/neuron_smFISH/masks/,neurite,Cf,Ago2,20220426_Ago2CfGFP-05_2022-04-26_Sam_smifish_C...


In [335]:
area_csv.to_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/area_sizes_2.csv', index=False)  


In [336]:
#area_csv['id'] = area_csv["filename"].str.partition(spl_word)[0]
#area_csv.head(5) ['gene', 'area', 'structure', 'shRNA']

#area_id = area_csv.filter(['area', 'id'], axis = 1)

#area_id.head(5)

#[['id', 'area', 'structure']]

In [337]:
area_id = area_csv.filter(['area', 'id', 'gene', 'structure'], axis = 1)
area_id.head(5)

,area,id,gene,structure
0,477395.0,20220426_Ago2CfGFP-01_2022-04-26_Sam_smifish_C...,Cf,neurite
1,263793.0,20220426_Ago2CfGFP-02_2022-04-26_Sam_smifish_C...,Cf,neurite
2,0.0,20220426_Ago2CfGFP-03_2022-04-26_Sam_smifish_C...,Cf,neurite
3,251333.0,20220426_Ago2CfGFP-04_2022-04-26_Sam_smifish_C...,Cf,neurite
4,127864.0,20220426_Ago2CfGFP-05_2022-04-26_Sam_smifish_C...,Cf,neurite


In [338]:
df3 = df.merge(area_csv[['id', 'area', 'structure']], on = ['id', 'structure'], how = 'outer')
df3.head(5)


,filename,folder,target,count,structure,gene,shRNA,id,area
0,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,194.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,24140.0
1,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-1_soma,NaN,45.0,soma,Map2,HBS1L,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,24140.0
2,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,867.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,96903.0
3,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-1_soma,NaN,197.0,soma,Map2,HBS1L,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,96903.0
4,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,379.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,72242.0


In [339]:
df3['smFISH_per_area'] = df3['count']/df3['area']
df3.head(5)

,filename,folder,target,count,structure,gene,shRNA,id,area,smFISH_per_area
0,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,194.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,24140.0,0.008036
1,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-1_soma,NaN,45.0,soma,Map2,HBS1L,20211123_smiFISH_HBS1LshRNA_01_2021-11-23_sam-...,24140.0,0.001864
2,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,867.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,96903.0,0.008947
3,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-1_soma,NaN,197.0,soma,Map2,HBS1L,20211123_smiFISH_HBS1LshRNA_02_2021-11-23_sam-...,96903.0,0.002033
4,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,Hbs1l_Map2endogenous/smFISH-2_soma,NaN,379.0,soma,GAPDH,HBS1L,20211123_smiFISH_HBS1LshRNA_03_2021-11-23_sam-...,72242.0,0.005246


In [340]:
len(df3)

284

In [341]:
df3.to_csv('/Users/lbreima/Documents/PhD_local/Manuscripts/2021_Sam_zipcodes/database/total_database.csv', index=False)  